# Tutorial: CUDS API

This tutorial introduces the CUDS API. The code given here is based on [this](https://github.com/simphony/osp-core/blob/master/examples/api_example.py) example.

## Background

CUDS stands for Common Universal Data Structure, and it is used to uniformly represent ontology individuals. In the python implementation of OSP-core, it means that every ontology individual is an instance of the `Cuds` class.

As every CUDS object is an ontology individual, each CUDS is related to an ontology class via the ontological `is a` relation, and can be connected to other CUDS objects through ontology relationships. In the python implementation of OSP-core, all such ontological concepts are instances of the `OntologyClass` class, which is itself a subclass of `OntologyEntity`. The ontology entitites are organized in namespaces.

In OSP-core, the ontology relationships can be tagged as active or passive relationships. This is done in the ontology installation file. Such feature lets CUDS objects act as containers, so that content of a CUDS object consists of other CUDS objects. This means that a CUDS is a [recursive data structure](https://en.wikipedia.org/wiki/Recursive_data_type). If a CUDS is connected via an active relationship to another CUDS, then the former contains the latter, while when a CUDS is connected via a passive relationship to another CUDS, the latter is contained in the former. Untagged ontology relationships do not define any containment.

[//]: # (TODO: Image of this containment idea, necessary to express the idea of the directionality of the relationships as well.)

The most important functionalities that the CUDS data structure exposes as python methods are the following:
- **add**: Connects the current CUDS object to one or more CUDS objects through a specific ontology relationship.
- **remove**: Despite its name, it does NOT delete the CUDS object itself. Instead, it just disconnects the current CUDS object from one or more CUDS objects.
- **get**: Returns the CUDS objects connected to the current CUDS object through a specific ontology relationship. 
- **iter**: Similar to the `get` method, it just returns one CUDS objects at a time instead of all at once (python iterator), so that memory can be saved.
- **is_a**: Checks if the CUDS object is an instance of the given oclass.

In addition, other important functionalities are exposed as python properties:
- **oclass**: The ontology class of the ontology individual represented by the CUDS object. If the individual belongs to multiple classes, only one of them is referenced by the property.
- **uid**: A unique ID identifying an ontology individual.
- **iri**: The [Internationalized Resource Identifier](https://fusion.cs.uni-jena.de/fusion/blog/2016/11/18/iri-uri-url-urn-and-their-differences/) of the CUDS object. It consists of a CUDS namespace prefix and the unique ID of the CUDS object. This will be clarified in the tutorial.
- **attributes**: The values of the ontology attributes of an individual (also known as [data properties](https://www.w3.org/TR/owl2-syntax/#Data_Properties) in [OWL](https://en.wikipedia.org/wiki/Web_Ontology_Language)) may also be accessed as python properties of the CUDS objects. For example: `cuds_object.name`.

There are some advanced functionalities NOT covered in this tutorial. Among them, we highlight the **update** method, which is covered in the [wrapper tutorial](multiple-wrappers.ipynb), where it can be seen in action. For a complete list of available methods and properties, check the API reference. That whole set of methods and attributes constitutes the CUDS [API](https://en.wikipedia.org/wiki/API).

[//]: # (TODO: Link to API reference.)

## Let's get hands on
In this tutorial, we will work with the `city` namespace, the example namespace from OSP-core. It consists of concepts from the example [city ontology](https://github.com/simphony/osp-core/blob/master/osp/core/ontology/docs/city.ontology.yml). There is [another tutorial](#) on the ontology YAML installation file. However, it is not important for the purposes of this tutorial.

[//]: # (TODO: simple graphical representation of the city ontology.)

The first step is to install the city ontology. Use the tool pico for this:

In [1]:
!pico install city

INFO 2021-03-30 12:03:17,518 [osp.core.ontology.installation]: Will install the following namespaces: ['city']
INFO 2021-03-30 12:03:17,536 [osp.core.ontology.yml.yml_parser]: Parsing YAML ontology file /home/jose/.local/lib/python3.9/site-packages/osp/core/ontology/docs/city.ontology.yml
INFO 2021-03-30 12:03:17,569 [osp.core.ontology.yml.yml_parser]: You can now use `from osp.core.namespaces import city`.
INFO 2021-03-30 12:03:17,569 [osp.core.ontology.parser]: Loaded 202 ontology triples in total
INFO 2021-03-30 12:03:17,589 [osp.core.ontology.installation]: Installation successful


Then you can import the `city` namespace.

In [2]:
# If you just installed the ontology from within this notebook and this line doesn't work, please restart the kernel and run this cell again.
from osp.core.namespaces import city

You are now creating some CUDS objects that you are going to use to try out the functionalities of the CUDS data structure.

In [3]:
c = city.City(name="Freiburg", coordinates=[47, 7])  # Ontology individual representing the city of Freiburg.
p1 = city.Citizen(name="Peter")  # Ontology indidual representing a specific person, "Peter".
p2 = city.Citizen(name="Anne")  # Ontology individual representing another specific person, "Anne".

The names `c, p1, p2` are assigned to the newly created CUDS objects. The keyword arguments `name` and `coordinates` let you directly assign values for such ontology attributes (also known as [data properties](https://www.w3.org/TR/owl2-syntax/#Data_Properties) in [OWL](https://en.wikipedia.org/wiki/Web_Ontology_Language)) to the new CUDS objects. The available ontology attributes for each ontology class depend on the specific class being instantiated. For example, the ontology attribute `name` is available for both the _City_ and the _Citizen_ ontology classes in the sample _City_ ontology. The attribute `coordinates` is available for the _City_ ontology class, but not for the _Citizen_ class.

### Functionalities exposed as python properties

Each CUDS object has a unique identifier (UID), which can be accessed using the `uid` property:

In [4]:
print("uid of c: " + str(c.uid))
print("uid of p1: " + str(p1.uid))
print("uid of p2: " + str(p2.uid))

uid of c: bc274beb-04ab-488f-aed0-550b6a433256
uid of p1: db7873e2-668c-424c-b7d8-b388968a060a
uid of p2: 55149899-c4b5-4858-96ea-c78921cca90e


Similarly, each CUDS object has [IRI](https://fusion.cs.uni-jena.de/fusion/blog/2016/11/18/iri-uri-url-urn-and-their-differences/), which serves to reference it in the [Semantic Web](https://en.wikipedia.org/wiki/Semantic_Web) and improves the compatibility of the CUDS format with the [Resource Description Framework](https://en.wikipedia.org/wiki/Resource_Description_Framework) data model. Note that the IRI of each CUDS object contains its unique identifier.

In [5]:
print("IRI of c: " + str(c.iri))
print("IRI of p1: " + str(p1.iri))
print("IRI of p2: " + str(p2.iri))

IRI of c: http://www.osp-core.com/cuds#bc274beb-04ab-488f-aed0-550b6a433256
IRI of p1: http://www.osp-core.com/cuds#db7873e2-668c-424c-b7d8-b388968a060a
IRI of p2: http://www.osp-core.com/cuds#55149899-c4b5-4858-96ea-c78921cca90e


The class of the ontology individual repredsented by the CUDS object can be queried as well:

In [6]:
print("oclass of c: " + str(c.oclass))
print("oclass of p1: " + str(p1.oclass))
print("oclass of p2: " + str(p2.oclass))

oclass of c: city.City
oclass of p1: city.Citizen
oclass of p2: city.Citizen


Finally, the values of the ontology attributes of an individual can be easily accessed using the dot notation.

In [7]:
print(f"Name of c: {c.name}. Coordinates of c: {c.coordinates}." )
print("Name of p1: " + str(p1.name))
print("Name of p2: " + str(p2.name))

Name of c: Freiburg. Coordinates of c: [47  7].
Name of p1: Peter
Name of p2: Anne


### Functionalities exposed as python methods

Now, we may connect the two citizens to our city object:

In [8]:
c.add(p1, rel=city.hasInhabitant)
c.add(p2, rel=city.hasInhabitant)

<city.Citizen: 55149899-c4b5-4858-96ea-c78921cca90e,  CoreSession: @0x7f1f08091400>

Note that the relationship type between the city and its two citizens in this case is 'HAS_INHABITANT'. In our context, this means that Anne and Peter are Freiburg inhabitants. Moreover, in the [city ontology](https://github.com/simphony/osp-core/blob/master/osp/core/ontology/docs/city.ontology.yml), this relationship is defined as an active relationship. This means that Anne and Peter are not only connected to Freiburg, but are also contained in the Freiburg CUDS object.

Next, we would like to iterate over the objects contained in the city object. We do so by using the `iter` function:


In [9]:
for el in c.iter():
    print("uid: " + str(el.uid))

uid: db7873e2-668c-424c-b7d8-b388968a060a
uid: 55149899-c4b5-4858-96ea-c78921cca90e


We can `get` a target object from a CUDS object if we have a UID of one of its immediate contained objects. This will not work if the target object is not contained in the CUDS object, but just connected to it.

In [10]:
print(c.get(p1.uid))  # `p1`is contained in `c` because they are connected through an active relationship.
print(p1.get(c.uid))  # `c` is NOT contained in `p1`

city.Citizen: db7873e2-668c-424c-b7d8-b388968a060a
None


We can also filter the contained objects by type: 

In [11]:
print(c.get(oclass=city.Citizen))

[<city.Citizen: db7873e2-668c-424c-b7d8-b388968a060a,  CoreSession: @0x7f1f08091400>, <city.Citizen: 55149899-c4b5-4858-96ea-c78921cca90e,  CoreSession: @0x7f1f08091400>]


We remove objects using the `remove()` function. Despite its name, this just disconnects the target object from the CUDS, but does NOT delete the target object from the memory.

In [12]:
c.remove(p1)
# c.remove(p1.uid) also works!
print(p1)  # `p1`still exists,
print(c.get(p1.uid))  # but is no longer connected neither contained in `c`.

city.Citizen: db7873e2-668c-424c-b7d8-b388968a060a
None


Let's close this tutorial by adding some neighborhoods in a loop,

In [13]:
for i in range(6):
    c.add(city.Neighborhood(name="neighborhood %s" % i))

and then verifying that they are indeed neighborhoods, just to also try the `is_a` method.



In [14]:
all(n.is_a(city.Neighborhood) for n in c.get(oclass=city.Neighborhood))

True

The existing ontology individuals and the relationships among them at the end of the tutorial are depicted below. Note that some attributes that were not specified were set automatically to the default values specified in the ontology.

In [15]:
from osp.core.utils import pretty_print  # This is an utility, see the Utilities section https://simphony.readthedocs.io/en/latest/utils.html?highlight=pretty_print.
pretty_print(c)

- Cuds object named <Freiburg>:
  uuid: bc274beb-04ab-488f-aed0-550b6a433256
  type: city.City
  superclasses: city.City, city.GeographicalPlace, city.PopulatedPlace, cuba.Entity
  values: coordinates: [47  7]
  description: 
    To Be Determined

   |_Relationship city.hasInhabitant:
   | -  city.Citizen cuds object named <Anne>:
   |    uuid: 55149899-c4b5-4858-96ea-c78921cca90e
   |    age: 25
   |_Relationship city.hasPart:
     -  city.Neighborhood cuds object named <neighborhood 0>:
     .  uuid: 7118ec24-fd85-4a40-8158-07117945cc46
     .  coordinates: [0 0]
     -  city.Neighborhood cuds object named <neighborhood 1>:
     .  uuid: 3c195b61-d7c1-4b5f-8d3b-4fcc2b4c5964
     .  coordinates: [0 0]
     -  city.Neighborhood cuds object named <neighborhood 2>:
     .  uuid: fe59fcf6-55b0-4744-81c5-96330a9e2fa5
     .  coordinates: [0 0]
     -  city.Neighborhood cuds object named <neighborhood 3>:
     .  uuid: e9b2c368-370c-4ff9-8256-7e5d5ce941fe
     .  coordinates: [0 0]
     -  